# Exponential Map

In [ ]:
import numpy as np

from sympy.algebras.quaternion import Quaternion

from scipy.linalg import expm, logm

In [ ]:
# different brackets

def C(x,y):
    return x@y - y@x

def Q(x,y):
    return x*y - y*x

def R(x,y):
    return np.cross(x,y)-np.cross(y,x)

# initial terms of Baker-Campbell-Hausdorff
def baker(x,y,B):
    return x+y + B(x,y)/2 + B(x,B(x,y))/12 + B(y,B(x,y))/12

deg = np.pi/180

In [ ]:
np.set_printoptions(precision=3, suppress=True)

## Simple example in Aff(1)

An element of the group:

In [ ]:
m1 = np.array( [[1,2],[0,1]] )
m1

The corresponding element of the Algebra:

In [ ]:
t1 = logm(m1)
t1

Another element:

In [ ]:
m2 = np.array( [[4,3],[0,1]] )
t2 = logm(m2)

Composition in the group:

In [ ]:
m1 @ m2

The exponential property $e^Ae^B = e^{A+B}$ fails because the group is not commutative.

In [ ]:
expm(t1+t2)

The Baker-campbell-Hausdorff formula gives a good approximation:

In [ ]:
expm(baker(t1,t2,C))

In the other direction:

In [ ]:
m2 @ m1

In [ ]:
expm(baker(t2,t1,C))

## Example in SO(3)

In [ ]:
def rot3(ang):
    c = np.cos(ang)
    s = np.sin(ang)
    return np.array([[c, -s, 0]
                  ,[s,  c, 0]
                  ,[0,  0, 1]])

def rot1(ang):
    c = np.cos(ang)
    s = np.sin(ang)
    return np.array([[1, 0,  0]
                  ,[0, c, -s]
                  ,[0, s,  c]])

def rot2(ang):
    c = np.cos(ang)
    s = np.sin(ang)
    return np.array([[ c, 0, s]
                  ,[ 0, 1, 0]
                  ,[-s, 0, c]])

We repeat the same experiment in the group or 3D rotations:

In [ ]:
m1 = rot3(30*deg)
t1 = logm(m1)

m2 = rot1(40*deg)
t2 = logm(m2)

In [ ]:
m1

In [ ]:
t1

The matrix logarithm of the rotation gives the angle (see the last section).

In [ ]:
t1/deg

In [ ]:
m1 @ m2

In [ ]:
expm(t1+t2)

Again, we need the BCH formula to get a good approximation in the tangent space:

In [ ]:
expm(baker(t1,t2,C))

In [ ]:
m2 @ m1

In [ ]:
expm(baker(t2,t1,C))

We now check the represention of rotations by unit quaternions. First we define two elements and verify that they correspond to the expected matrices.

In [ ]:
u1 = np.array([0,0,1])
a1 = 30*deg
u2 = np.array([1,0,0])
a2 = 40*deg

q1 = Quaternion(np.cos(a1/2),*(np.sin(a1/2)*u1))
q2 = Quaternion(np.cos(a2/2),*(np.sin(a2/2)*u2))

In [ ]:
q1

In [ ]:
q1.to_axis_angle()

In [ ]:
q1.to_rotation_matrix()

In [ ]:
m1

In [ ]:
q2

In [ ]:
q2.to_axis_angle()

In [ ]:
q2.to_rotation_matrix()

In [ ]:
m2

Then we create the composition $q_2q_1$ and obtain the rotation matrix by conjugation of the basis:

In [ ]:
q = q2*q1

np.array([(q*Quaternion(0,*b)*q.inverse()).args[1:] for b in np.eye(3)]).T.astype(float)

In [ ]:
m2@m1

In [ ]:
q.to_rotation_matrix()

Now check the approximation in the tangent space:

In [ ]:
tq1=Quaternion(0,0,0,a1/2)
tq2=Quaternion(0,a2/2,0,0)

First we check that the elements of the algebra, which are pure imaginary quaternions, are correct (it seems that the quaternion logarithm is not implemented). This is the quaternion exponential:

In [ ]:
tq1.exp()

Then check the BCH formula:

In [ ]:
baker(tq2,tq1,Q)

In [ ]:
baker(tq2,tq1,Q).exp()

We get the expected good approximation:

In [ ]:
baker(tq2,tq1,Q).exp().to_rotation_matrix()

In [ ]:
m2@m1

Finally, we check that the Lie Bracket is equivalent to the standard cross product:

In [ ]:
v1 = np.array(tq1.args[1:]).astype(float)
v2 = np.array(tq2.args[1:]).astype(float)

In [ ]:
baker(v2,v1,R)

In [ ]:
baker(tq2,tq1,Q)

## Exponential and logarithm of rotations

In [ ]:
import sympy as sym

I = sym.I
def mat(x1,x2,x3,x4):
    return sym.Matrix([[x1,x2],[x3,x4]])

theta = sym.Symbol('theta',Real=True)
m = mat(0,-theta,theta,0)
m

In [ ]:
m.exp()

We explicitly compute the matrix exponential from the factorization.

In [ ]:
m.eigenvects()

In [ ]:
mat(-I,I,1,1) * mat(-I*theta,0,0,I*theta)* mat(-I,I,1,1).inv()

In [ ]:
r = mat(-I,I,1,1) * mat(sym.exp(-I*theta),0,0,sym.exp(I*theta))* mat(-I,I,1,1).inv()
r

In [ ]:
sym.simplify(r)